# Summarize datasets for all operator results
Code author: Sahar H. El Abbadi
Date started: 2023-03-13
Date last edited: 2023-03-17

## Generate summary tables

Generate overpass summary tables for each operator. Overpass summary tables extract all relevant information from operator reports and metering data that is needed for downstream analysis. These tables are saved as CSV files in 03_results > overpass_summary, using the format operator_stage_overpasses.csv.

Columns in the overpass summary tables are:
- overpass_id: this is the ID for the specific aircraft overpass, and matches PerformerExperimentID overpass number in the raw operator data
- overpass_datetime: date and time of overpass, in UTC time. This can be generated using the Flightradar GPS timestamp (timekeeper = 'flightradar'), Stanford's on the ground estimate of overhead time (timekeeper = 'stanford'), or using the timestamp according to the operator report (timekeeper = 'team'). Value used for all analysis is in paper is Flightradar
- zero_release: True if the release by Stanford is 0 kgh, False if greater than 0 kgh
- non_zero_release: True if release by Stanford is greater than 0 kgh, False if equal to 0 kgh
- operator_kept: True if this overpass passed the operator QC criteria
- stanford_kept: True if this overpass passed Stanford's QC criteria
- phase_iii: True if we provided this overpass to the operator during Phase III of unblinding
- pass_all_qc: True if passed both operator and Stanford QC
- fail_all_qc: True if this overpass failed both operator and Stanford QC
- operator_detected: True if operator detected a release. False if they did not
- operator_quantification: operator's quantification estimate as reported in operator report
- operator_lower: lower bound on operator's lower bound quantification estimate
- operator_upper: upper bound on operator's quantification estimate
- qc_summary: summarizes results of both operator and Stanford QC. Must be one of the following: 'pass_all', 'fail_stanford', 'fail_operator', 'fail_all'

Note: due to the small number of data points for Scientific Aviation and the fact that they did not submit using a template, sciav_1_overpasses.csv was manually constructed by Sahar El Abbadi on 2023-03-16.

In [1]:
# Imports
from methods_source import generate_all_overpass_reports

generate_all_overpass_reports(strict_discard = False, timekeeper = 'flightradar')
generate_all_overpass_reports(strict_discard = True, timekeeper = 'flightradar')

## Sanity checks on data

Basic double checks on the data to make sure everything looks good

##### Number of overpasses

Check that the length of the dataframe matches the highest overpass ID. This will flag the potential for missing or duplicate data in the summary file. Doing this check as we previously observed duplicate rows in the meter summary file.

In [2]:
# Check Carbon Mapper

from methods_source import load_overpass_summary, check_overpass_number

cm_overpasses = load_overpass_summary(operator='Carbon Mapper', stage=1, strict_discard=False)
cm_max_id = cm_overpasses.max()['overpass_id']
cm_overpasses_length = len(cm_overpasses)
check_overpass_number(operator='Carbon Mapper', max_overpass_id=cm_max_id, overpasses_length=cm_overpasses_length)

The length of the Carbon Mapper overpasses dataframe (121) matches the highest value for overpass_id (121).


In [3]:
# Check GHGSat

from methods_source import load_overpass_summary, check_overpass_number

ghg_overpasses = load_overpass_summary(operator='GHGSat', stage=1, strict_discard=False)
ghg_max_id = ghg_overpasses.max()['overpass_id']
ghg_overpasses_length = len(ghg_overpasses)
check_overpass_number(operator='GHGSat', max_overpass_id=ghg_max_id, overpasses_length=ghg_overpasses_length)

The maximum GHGSat overpass ID is less than the length of the overpass dataframe. Check for duplicate values in summary data.
Length of GHGSat overpasses dataframe: 198
Highest GHGSat overpass_id: 192


In [4]:
# Check Kairos

from methods_source import load_overpass_summary, check_overpass_number

kairos_overpasses = load_overpass_summary(operator='Kairos', stage=1, strict_discard=False)
kairos_max_id = kairos_overpasses.max()['overpass_id']
kairos_overpasses_length = len(kairos_overpasses)
check_overpass_number(operator='Kairos', max_overpass_id=kairos_max_id, overpasses_length=kairos_overpasses_length)

The length of the Kairos overpasses dataframe (349) matches the highest value for overpass_id (349).


In [5]:
# Check Scientific Aviation

from methods_source import load_overpass_summary, check_overpass_number

sciav_overpasses = load_overpass_summary(operator='Scientific Aviation', stage=1, strict_discard=False)
sciav_max_id = sciav_overpasses.max()['overpass_id']
sciav_overpasses_length = len(sciav_overpasses)
check_overpass_number(operator='Scientific Aviation', max_overpass_id=sciav_max_id, overpasses_length=sciav_overpasses_length)

The length of the Scientific Aviation overpasses dataframe (18) matches the highest value for overpass_id (18).


In [6]:
# Check Methane Air

#TODO check Methane Air

##### Calculate average flight altitude across all reported overpasses

Use "altitude_feet" column in overpass summary files

In [7]:
from methods_source import load_overpass_summary

cm_overpasses = load_overpass_summary(operator='Carbon Mapper', stage=1, strict_discard=False)
ghg_overpasses = load_overpass_summary(operator='GHGSat', stage=1, strict_discard=False)
kairos_overpasses = load_overpass_summary(operator='Kairos', stage=1, strict_discard=False)
cm_altitude = cm_overpasses.altitude_feet.mean()
ghg_altitude = ghg_overpasses.altitude_feet.mean()
kairos_altitude = kairos_overpasses.altitude_feet.mean()
#TODO add MAIR
print('Average overpass altitude (feet) across all reported overpasses:\n')
print(f'Carbon Mapper: {cm_altitude:,.1f} ft')
print(f'GHGSat: {ghg_altitude:,.1f} ft')
print(f'Kairos: {kairos_altitude:,.1f} ft')

Average overpass altitude (feet) across all reported overpasses:

Carbon Mapper: 11,749.1 ft
GHGSat: 8,003.4 ft
Kairos: 2,758.8 ft


##### Calculate average flight for each day of testing


In [8]:
from methods_source import calc_daily_altitude
operator = 'Carbon Mapper'
cm_alt = calc_daily_altitude(operator)
print(f'{operator} average daily altitude:\n')
print(cm_alt.to_string(formatters={'altitude_feet':'{:,.1f} feet'.format}))

Carbon Mapper average daily altitude:

           altitude_feet
2022-10-10 11,779.6 feet
2022-10-11 11,734.4 feet
2022-10-12 11,626.3 feet
2022-10-28 11,589.9 feet
2022-10-29 11,780.9 feet
2022-10-31 11,927.3 feet


In [9]:
operator = 'GHGSat'
ghg_alt = calc_daily_altitude(operator)
print(f'{operator} average daily altitude:\n')
print(ghg_alt.to_string(formatters={'altitude_feet':'{:,.1f} feet'.format}))

GHGSat average daily altitude:

           altitude_feet
2022-10-31  7,972.6 feet
2022-11-02  7,983.6 feet
2022-11-04  7,983.0 feet
2022-11-07  8,101.5 feet


In [10]:
operator = 'Kairos'
kairos_alt = calc_daily_altitude(operator)
print(f'{operator} average daily altitude:\n')
print(kairos_alt.to_string(formatters={'altitude_feet':'{:,.1f} feet'.format}))

Kairos average daily altitude:

           altitude_feet
2022-10-24  2,741.9 feet
2022-10-25  2,725.4 feet
2022-10-26  2,736.3 feet
2022-10-27  2,796.8 feet
2022-10-28  2,793.1 feet
